# Data processing

## 1.Downing allosteric site data from shsmu.edu.cn

> `data/allosteric_site_shsmu.json` 
> 
> contains data crawling form shsmu.edu.cn(home->featrue->site).

In [ ]:
# there are 1928 allosteric site in shsmu, but 1928th 3GVU can not download.

from utils.data_download import download_shsmu_as, download_rcsb, unzip

allosteric_site_index = 'data/allosteric_site_shsmu.json'

download_shsmu_as(allosteric_site_path=allosteric_site_index, outpath='data/shsmu_allosteric_site/')
unzip(origin_dir='data/shsmu_allosteric_site/', outdir='data/shsmu_allosteric_site/allosteric_site/')

!rm -r data/shsmu_allosteric_site/*.pdb.gz

download_rcsb(allosteric_site_path=allosteric_site_index, outpath='data/shsmu_allosteric_site/')

!rm -r data/shsmu_allosteric_site/3gvu.pdb.gz

unzip(origin_dir='data/shsmu_allosteric_site/', outdir='data/shsmu_allosteric_site/rscb_pdb/')

!rm -r data/shsmu_allosteric_site/*.pdb.gz

> `data/ASD_Release_201909_AS.txt` 
>
> is the description file of the allostericsite, but hard to read.

In [ ]:
from utils.data_process import transform_txt_to_csv

txt_path = 'data/ASD_Release_201909_AS.txt'
csv_path = 'data/ASD_Release_201909_AS.csv'
transform_txt_to_csv(path=txt_path, outpath=csv_path)

## 2.Download pretain data form rcsb

> `data/pretrain/list_file_protein_xray_max3A_total.txt`
>
> includes pdbids with resolution of 0.5 ~ 3A obtained by X-Ray experiment.

In [ ]:
# download logs are stored in succeed.log, timeout.log, wrang.log

from utils.data_download import download_pretarining_data, redownload_error_pretarining_data

pdblist = 'data/pretrain/list_file_protein_xray_max3A_total.txt'
output_path = 'data/pretrain/'

download_pretarining_data(list_path=pdblist, outpath=output_path)

> If there are links in timeout.log or wrang.log run redownload_error_pretarining_data() to redownload failed pdbs.

In [ ]:
timeout_path = 'data/pretrain/timeout.log'
wrang_path = 'data/pretrain/wrang.log'

redownload_error_pretarining_data(timeout_path, wrang_path, output_path)